In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install --upgrade torch==1.5.0
!pip install fairseq==0.9.0
!pip install pytorch-lightning
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 306 kB 5.2 MB/s 
     |████████████████████████████████| 90 kB 8.8 MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp37-cp37m-linux_x86_64.whl size=2155897 sha256=b595a52aa41ef33dc4e5e50810696e141be7c2aa9d1f9fa04144df3d135e5448
  Stored in directory: /root/.cache/pip/wheels/6b/27/e2/c55614da7eb71041bb08f02e8a302b869e51185eb7c575a604
Successfully built fairseq
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 132 kB 47.6 MB/s 
     |████████████████████████████████| 829 kB 36.2 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 329 kB 49.8 MB/s 
     |████████████████████████████████| 1.1 MB 44.2 MB/s 
     |████████████████████████████████| 160 kB 48.3 MB/s 
     |████████████████████████████████| 271 kB 47.6 MB/s 
     |████████████████████████████████| 192 kB 51.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl

# LOAD DATA

In [ ]:
DATA_ROOT_DIR="/content/drive/MyDrive/programming/nlp/dataset/shopee-sentiment"
!ls $DATA_ROOT_DIR

sample_submission.csv  train.csv		       train.xlsx
test.csv	       train_preprocess.csv
test_preprocess.csv    train_preprocess_unsegment.csv


In [ ]:
import pandas as pd
import numpy as np
from torch.utils.data import random_split, DataLoader, Dataset
import pytorch_lightning as pl
import torch.nn as nn
import torch

train_ratio = 0.8
DATA_DIR = DATA_ROOT_DIR + '/train_preprocess_unsegment.csv'

In [ ]:
train = pd.read_csv(DATA_DIR, index_col=0)
train.head()

,id,text,class,preprocess_text
0,dee6dfc5,"Đến quán 2 lần thôi , rất là thích !\nQuán tuy...",1,đến quán 2 lần thôi rất là thích quán tuy nằm ...
1,800813f5,Đến quán vào tối chủ_nhật . Có band hát . Khá ...,0,đến quán vào tối chủ nhật có band hát khá ổn t...
2,6553e47f,Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi g...,0,phục vụ lâu quá mặc dù khách rất vắng đợi gần ...
3,b45a1ff1,"Ko gian bé_tí , quán chật_chội , đông người nê...",0,ko gian bé tí quán chật chội đông người nên ...
4,f92694b0,"Khi mình order , đặt bánh thì nhận được sự tiế...",1,khi mình order đặt bánh thì nhận được sự tiếp ...


In [ ]:
from typing import *

class SentimentData(Dataset):
  def __init__(self, data_dir):
    self.df = pd.read_csv(data_dir, index_col=0)

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    text = self.df["preprocess_text"][idx]
    label = self.df["class"][idx]
    
    return text, label

class SentimentDataModule(pl.LightningDataModule):

    def __init__(self, data_dir: str = DATA_DIR, batch_size: int = 16):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage: Optional[str] = None):
        data_full = SentimentData(self.data_dir)
        train_size = round(len(data_full)*train_ratio)
        val_size = len(data_full) - train_size
        print(len(data_full), train_size, val_size)
        self.data_train, self.data_val = random_split(data_full, [train_size, val_size])

    def train_dataloader(self):
        return DataLoader(self.data_train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.data_val, batch_size=self.batch_size)


if __name__ == "__main__":
	dm = SentimentDataModule(DATA_DIR)
	dm.setup()
	idx = 0
	for item in (dm.train_dataloader()):
		print(idx)
		print(item)
		idx += 1
		if idx > 5: break

27000 21600 5400
0
[('hôm nay ghé lại urban có thấy ra mắt 3 món mới về trà ấn tượng đầu tiên là mấy cái tên twilight lorax đồ cơ mà ngon lạ hợp với thời tiết hiện nay có điều hút mấy cái thạch trong ly lên khó quá có một món nóng nhưng chưa thử lần sau nhất định sẽ ghé lại d', 'là 1 nơi yên tĩnh và lịch sự rất hợp với cặp đôi hẹn hò hay canh chị em tâm sự họp nhóm đông thì có vẻ khôg hợp lắm vì không gian kín làm ồn kì lắm li socola đá say giá 48 rất ngon nhưng hơi ngọt nếu không thích uống ngọt nên kêu cho ít đường nha', 'helo san fu lou   mình muốn complain về nhân viên host waiter của nhà hàng mình không nói tên nhân viên vì chỉ muốn góp ý tối nay mình đi ăn với bạn gái nhà hàng còn khá nhiều chỗ nhưng đã có khách đặt còn cả tiếng đồng hồ nữa khách mới đến mình có nói rõ vs nhân viên host là mình chỉ ăn tối nhanh rồi đi vậy mà nhân viên host cứ từ chối rồi đổi chỗ ngồi lung tung sau khi mình order 2 món mới ăn được 25 trong khi vẫn còn 57 bàn trống mình tính gọi thêm món nhưng phục

In [ ]:
# !!git clone https://github.com/pytorch/fairseq
# %cd /content/fairseq/
# !python -m pip install --editable .
# python setup.py build_ext --inplace
# %cd /content

# Model

In [ ]:
from fairseq.data import Dictionary
import sentencepiece as spm
from os.path import join as pjoin
from transformers import PreTrainedTokenizer
import sentencepiece as spm

class XLMRobertaTokenizer(PreTrainedTokenizer):
    def __init__(
        self,
        pretrained_file,
        bos_token="<s>",
        eos_token="</s>",
        sep_token="</s>",
        cls_token="<s>",
        unk_token="<unk>",
        pad_token="<pad>",
        mask_token="<mask>",
        **kwargs
    ):
        super().__init__(
            bos_token=bos_token,
            eos_token=eos_token,
            unk_token=unk_token,
            sep_token=sep_token,
            cls_token=cls_token,
            pad_token=pad_token,
            mask_token=mask_token,
            **kwargs,
        )
        # load bpe model and vocab file
        sentencepiece_model = pjoin(pretrained_file, 'sentencepiece.bpe.model')
        vocab_file = pjoin(pretrained_file, 'dict.txt')
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(sentencepiece_model)   # please dont use anything from sp_model bcz it makes everything goes wrong
        self.bpe_dict = Dictionary().load(vocab_file)
        # Mimic fairseq token-to-id alignment for the first 4 token
        self.fairseq_tokens_to_ids = {"<s>": 0, "<pad>": 1, "</s>": 2, "<unk>": 3}
        # The first "real" token "," has position 4 in the original fairseq vocab and position 3 in the spm vocab
        self.fairseq_offset = 0
        self.fairseq_tokens_to_ids["<mask>"] = len(self.bpe_dict) + self.fairseq_offset
        self.fairseq_ids_to_tokens = {v: k for k, v in self.fairseq_tokens_to_ids.items()}
     
    def _tokenize(self, text): 
        return self.sp_model.EncodeAsPieces(text)

    def _convert_token_to_id(self, token):
        """ Converts a token (str) in an id using the vocab. """
        if token in self.fairseq_tokens_to_ids:
            return self.fairseq_tokens_to_ids[token]
        spm_id = self.bpe_dict.index(token)
        return spm_id

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (str) using the vocab."""
        if index in self.fairseq_ids_to_tokens:
            return self.fairseq_ids_to_tokens[index]
        return self.bpe_dict[index]

    @property
    def vocab_size(self):
        return len(self.bpe_dict) + self.fairseq_offset + 1  # Add the <mask> token

    def get_vocab(self):
        vocab = {self.convert_ids_to_tokens(i): i for i in range(self.vocab_size)}
        vocab.update(self.added_tokens_encoder)
        return vocab

In [ ]:
from transformers import XLMRobertaConfig, XLMRobertaForSequenceClassification
import torch

pretrained_path = '/content/drive/MyDrive/programming/nlp/pretrained/envibert/'
!ls $pretrained_path

roberta = XLMRobertaForSequenceClassification.from_pretrained(pretrained_path)
tokenizer = XLMRobertaTokenizer(pretrained_path)

config.json  dict.txt  model.pt  pytorch_model.bin  sentencepiece.bpe.model


You are using a model of type roberta to instantiate a model of type xlm-roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/drive/MyDrive/programming/nlp/pretrained/envibert/ were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassificat

In [ ]:
inputs = ["Tôi ghét nó", "Tôi thích nó", "Tôi quý nó"]
inputs = tokenizer(inputs, return_tensors='pt')
print(inputs)
outputs = roberta(**inputs, labels=torch.tensor([0, 1, 1]))
print(outputs)

{'input_ids': tensor([[ 842, 8919,  543],
        [ 842,  648,  543],
        [ 842,  976,  543]]), 'token_type_ids': tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]])}
SequenceClassifierOutput(loss=tensor(0.6879, grad_fn=<NllLossBackward>), logits=tensor([[-0.0371,  0.0014],
        [-0.0387, -0.0014],
        [-0.0375, -0.0037]], grad_fn=<AddmmBackward>), hidden_states=(tensor([[[-5.0629e-02, -1.3749e-01, -1.0452e-01,  ..., -5.6184e-03,
           7.6252e-02, -7.4683e-02],
         [-5.0720e-02, -1.9042e-01, -2.5382e-01,  ..., -2.4447e-01,
          -9.7407e-02, -1.1625e-02],
         [ 2.4340e-01, -7.1564e-02,  2.5017e-01,  ..., -2.3644e-01,
           1.2962e-01,  2.3541e-01]],

        [[-5.0629e-02, -1.3749e-01, -1.0452e-01,  ..., -5.6184e-03,
           7.6252e-02, -7.4683e-02],
         [ 1.0403e-01, -1.9165e-01, -5.2380e-01,  ..., -2.7758e-01,
          -1.0180e-01, -4.1142e-02],
         [ 2.

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score

class SentimentRoberta(pl.LightningModule):
  def __init__(self, lr_roberta, lr_classifier):
    super().__init__()
    self.roberta = XLMRobertaForSequenceClassification.from_pretrained(pretrained_path)
    self.tokenizer = XLMRobertaTokenizer(pretrained_path)
    self.lr_roberta = lr_roberta
    self.lr_classifer = lr_classifier


  def forward(self, texts, labels=None):
    inputs = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=256)
    for key in inputs:
      inputs[key] = inputs[key].to(self.device)

    outputs = self.roberta(**inputs, labels=labels)
    return outputs


  def configure_optimizers(self):
    roberta_params = self.roberta.roberta.named_parameters()
    classifier_params = self.roberta.classifier.named_parameters()

    grouped_params = [
      {"params": [p for n, p in roberta_params], "lr": self.lr_roberta},
      {"params": [p for n, p in classifier_params], "lr": self.lr_classifer}
    ]
    optimizer = torch.optim.AdamW(
        grouped_params
    )
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.98)
    return {
        'optimizer': optimizer,
        'lr_scheduler': {
            'scheduler': scheduler,
            'monitor': 'f1/val',
        }
    }

  def training_step(self, batch, batch_idx):
    texts, labels = batch
    outputs = self(texts, labels=labels)

    if len(outputs.values()) == 3:
      loss, logits, _ = outputs.values()
    else:
      loss, logits = outputs.values()
    return loss

  def validation_step(self, batch, batch_idx):
    texts, labels = batch
    outputs = self(texts, labels=labels)

    if len(outputs.values()) == 3:
      loss, logits, _ = outputs.values()
    else:
      loss, logits = outputs.values()

    output_scores = torch.softmax(logits, dim=-1)
    return loss, output_scores, labels

  def validation_epoch_end(self, validation_step_outputs):

    val_preds = torch.tensor([], device=self.device)
    val_scores = torch.tensor([], device=self.device)
    val_labels = torch.tensor([], device=self.device)
    val_loss = 0
    total_item = 0

    for idx, item in enumerate(validation_step_outputs):
      loss, output_scores, labels = item

      predictions = torch.argmax(output_scores, dim=-1)
      val_preds = torch.cat((val_preds, predictions), dim=0)
      val_scores = torch.cat((val_scores, output_scores[:,1]), dim=0)
      val_labels = torch.cat((val_labels, labels), dim=0)

      val_loss += loss
      total_item += 1

    # print("VAL PREDS", val_preds.shape)
    # print("VAL SCORES", val_scores.shape)
    # print("VAL LABELS", val_labels.shape)
    val_preds = val_preds.cpu().numpy()
    val_scores = val_scores.cpu().numpy()
    val_labels = val_labels.cpu().numpy()
    
    reports = classification_report(val_labels, val_preds, output_dict=True)
    print("VAL LABELS", val_labels)
    print("VAL SCORES", val_scores)
    try:
      auc = roc_auc_score(val_labels, val_scores)
    except Exception as e:
      print(e)
      print("Cannot calculate AUC. Default to 0")
      auc = 0
    accuracy = accuracy_score(val_labels, val_preds)

    print(classification_report(val_labels, val_preds))

    self.log("loss/val", val_loss)
    self.log("auc/val", auc)
    self.log("accuracy/val", accuracy)
    self.log("precision/val", reports["weighted avg"]["precision"])
    self.log("recall/val", reports["weighted avg"]["recall"])
    self.log("f1/val", reports["weighted avg"]["f1-score"])


In [ ]:
trainer = pl.Trainer(
    fast_dev_run=True,
)
model = SentimentRoberta(lr_roberta=1e-5, lr_classifier=3e-3)
dm = SentimentDataModule()

trainer.fit(model, dm)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1567: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
You are using a model of type roberta to instantiate a model of type xlm-roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/drive/MyDrive/programming/nlp/pretrained/envibert/ were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initiali

27000 21600 5400


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:395: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:57: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 959. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:57: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 547. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


VAL LABELS [1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1.]
VAL SCORES [0.11248375 0.14095211 0.09060212 0.13426316 0.14714074 0.095732
 0.09746041 0.1130011  0.11765185 0.10690454 0.13107814 0.12104373
 0.14510155 0.07852908 0.09756076 0.1506509 ]
              precision    recall  f1-score   support

         0.0       0.44      1.00      0.61         7
         1.0       0.00      0.00      0.00         9

    accuracy                           0.44        16
   macro avg       0.22      0.50      0.30        16
weighted avg       0.19      0.44      0.27        16



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training

In [ ]:
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

torch.manual_seed(123)

tb_logger = pl_loggers.TensorBoardLogger('/content/drive/MyDrive/colab/tb_logs/')

trainer = pl.Trainer(
    min_epochs=1,
    max_epochs=5,
    gpus=1,
    precision=16,
    val_check_interval=0.5,
    # check_val_every_n_epoch=1,
    callbacks=[
      ModelCheckpoint(
          dirpath='/content/drive/MyDrive/ckpt',
          save_top_k=3,
          monitor='f1/val',
      ), 
      EarlyStopping('f1/val', patience=5)
    ],
    fast_dev_run=False,
    logger=tb_logger
)

dm.setup(stage="fit")
trainer.fit(model, dm)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                                | Params
----------------------------------------------------------------
0 | roberta | XLMRobertaForSequenceClassification | 70.7 M
----------------------------------------------------------------
70.7 M    Trainable params
0         Non-trainable params
70.7 M    Total params
141.409   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:57: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 348. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


VAL LABELS [1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0.
 1. 1. 0. 1. 1. 1. 0. 0.]
VAL SCORES [0.1124538  0.14098547 0.09057447 0.13437785 0.14718705 0.09577148
 0.09756223 0.11284428 0.1176228  0.10697014 0.13106197 0.12111288
 0.1450548  0.07850098 0.09756223 0.15065254 0.15533455 0.09268778
 0.1127954  0.10622612 0.09773432 0.12683924 0.10194652 0.08632348
 0.08701921 0.15065254 0.18308188 0.09442688 0.08866235 0.13039611
 0.12080135 0.13609089]
              precision    recall  f1-score   support

         0.0       0.41      1.00      0.58        13
         1.0       0.00      0.00      0.00        19

    accuracy                           0.41        32
   macro avg       0.20      0.50      0.29        32
weighted avg       0.17      0.41      0.23        32



Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

VAL LABELS [1. 1. 0. ... 1. 1. 1.]
VAL SCORES [0.6376515  0.94402224 0.6559424  ... 0.9641868  0.9604726  0.9566342 ]
              precision    recall  f1-score   support

         0.0       0.95      0.85      0.90      2654
         1.0       0.87      0.96      0.91      2746

    accuracy                           0.90      5400
   macro avg       0.91      0.90      0.90      5400
weighted avg       0.91      0.90      0.90      5400



Validating: 0it [00:00, ?it/s]

VAL LABELS [1. 1. 0. ... 1. 1. 1.]
VAL SCORES [0.97879386 0.9685601  0.69014156 ... 0.9965436  0.9968149  0.9914391 ]
              precision    recall  f1-score   support

         0.0       0.96      0.83      0.89      2654
         1.0       0.85      0.97      0.91      2746

    accuracy                           0.90      5400
   macro avg       0.91      0.90      0.90      5400
weighted avg       0.91      0.90      0.90      5400



/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  f"DataModule.{name} has already been called, so it will not be called again. "


TEST

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/colab/tb_logs/'

In [ ]:
inputs = ["Món này ăn chán vậy"]
outputs = model(inputs)
logits = outputs['logits']
score = torch.softmax(logits, dim=-1)
print(score)
print(torch.argmax(score, dim=-1).item())


tensor([[0.8693, 0.1307]], grad_fn=<SoftmaxBackward>)
0


In [ ]:
!tensorboard dev upload --logdir '/content/drive/MyDrive/colab/tb_logs/'


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

/content/drive/MyDrive/colab/tb_logs/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=mWMILGxMaPys0LiDc2tsFTQdwBCzRH&prompt=con